In [6]:
import mlrun
import os
mlrun.__version__

'1.6.1'

In [22]:
project_name = "nodegroup-test"

project = mlrun.get_or_create_project(project_name, context="./")
git_source =              "git://github.com/xsqian/mlrun-github-example.git#main"
project.set_source(source=git_source, pull_at_runtime=True)
project.save()

> 2024-06-07 16:37:00,229 [info] Project loaded successfully: {'project_name': 'nodegroup-test'}


In [23]:
%%writefile functions/nodegroup_test.py

import time
import psutil


def nodegroup_test(context):
    context.logger.info("Resource Test!")
    mem = psutil.virtual_memory()
    vcpus = psutil.cpu_count(logical=True)
    pcpus = psutil.cpu_count(logical=False)
    context.logger.info(f"CPUs virtual  -> {vcpus}")
    context.logger.info(f"CPUs physical -> {pcpus}")
    context.logger.info(
        f"Memory avlbl  -> {round(mem.available/(1024 * 1024 * 1024), 2)}GB"
    )
    context.logger.info(f"Memory used   -> {round(mem.used/(1024 * 1024 * 1024), 2)}GB")
    context.logger.info(f"Sleeping 300s")


Overwriting functions/nodegroup_test.py


In [24]:
func2 = project.set_function(
    name="nodegroup-test",
    func="functions/nodegroup_test.py",
    kind="job",
    image="mlrun/mlrun:1.6.1",
    handler="nodegroup_test",
    with_repo=True,
)
func2.with_requests(mem="16G", cpu=2)
func2.with_limits(mem="32G", cpu=4)
func2.save()

'db://nodegroup-test/nodegroup-test:latest'

In [26]:
project.run_function(func2, local=True)

> 2024-06-07 16:37:35,458 [info] Storing function: {'name': 'nodegroup-test-nodegroup-test', 'uid': 'bf4cfe01bd0e4785b6c645127218fe8a', 'db': 'http://mlrun-api:8080'}
> 2024-06-07 16:37:35,583 [info] Resource Test!
> 2024-06-07 16:37:35,585 [info] CPUs virtual  -> 16
> 2024-06-07 16:37:35,586 [info] CPUs physical -> 8
> 2024-06-07 16:37:35,586 [info] Memory avlbl  -> 47.57GB
> 2024-06-07 16:37:35,586 [info] Memory used   -> 11.44GB
> 2024-06-07 16:37:35,587 [info] Sleeping 300s


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
nodegroup-test,...7218fe8a,0,Jun 07 16:37:35,completed,nodegroup-test-nodegroup-test,v3io_user=xingshengkind=localowner=xingshenghost=jupyter-xingsheng-5f5f9b5c46-mrvm5,,,,


> 2024-06-07 16:37:35,666 [info] Run execution finished: {'status': 'completed', 'name': 'nodegroup-test-nodegroup-test'}


In [12]:
%%writefile workflow/workflow.py
""" The main workflow of the project

This file creates the MLRun pipeline to run the functions
in an orchestrated manner

"""

from kfp import dsl
import mlrun


# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(name="testing-resource")
def test():
    run2 = mlrun.run_function(
        name="nodegroup-test",
        function="nodegroup-test",
        local=False,
    )

Writing workflow/workflow.py


In [13]:
project.set_workflow(
    name="workflow", workflow_path="workflow/workflow.py", handler="test"
)
project.save()

In [ ]:
project.run(
    name="workflow",
    schedule="40 22 * * 0-4",
    dirty=True,
)